In [1]:
from Bio import SeqIO
from Bio.Alphabet import generic_dna
from Bio.Seq import Seq
import csv
import re
import math

In [9]:
# This function imports the top barcodes from a comma separated (csv) file that contains your full primers.
# The format should be: 'primer name', 'NKKNKK-barcode-primer region'
# If your file only includes the barcodes and not the rest of the primer, you'll need to change the function,
# which I describe in the next comment.
def csv_barcode_import(barcode_file):
    with open(barcode_file) as infile:
        top_barcodes= {}
        reader=csv.reader(infile)
        # The second part of this expression (rows[1][6:16]) takes the first 10 characters following
        # the 6 character adapted ligation sequence (NKKNKK) and uses them as your barcode.
        # If you want to search for a shorter or longer sequence, add or substract from 16.
        mydict={rows[0]:rows[1][6:16] for rows in reader}
        for k,v in mydict.items():
            # I only include primers with 'top' in the name to 'clean up' my file so that it only contains 
            # correctly oriented reads. 
            # If your file only contains top barcodes anyway, or if you use 'forward' instead of top,
            # change the 'if' criteria below.
            if 'A3-top' in k:
                top_barcodes[k]= v
            if 'C2-top' in k:
                top_barcodes[k]= v
            if 'ns-top' in k:
                top_barcodes[k]= v
        return top_barcodes

In [21]:
top_barcode_list= csv_barcode_import('Primer_file.csv')
print(top_barcode_list)

{'K234X-S237X-ns-top': 'GTGGTATGCG', 'K234X-S237X-A3-top': 'CACATAGGCG', 'K234X-S237X-C2-top': 'CCTAAACTCG', 'K234X-T235X-ns-top': 'AGGAGTCCCG', 'K234X-T235X-A3-top': 'TCTACTCTCG', 'K234X-T235X-C2-top': 'TCCTGAGCCG'}


In [11]:
# end pairing is performed by first assuming each read in FASTQ file 1 is the top read,
# then assuming that each read in FASTQ file 2 is the top read
# reads are then filtered for any forward barcode, eliminating ends that were paired in the wrong direction.

#joins files assuming all reads from FASTQ file 1 are the forward read
def join_files_1fwd(file1, file2):
    with open(file1) as f1:
        with open(file2) as f2:
            with open('joined_1fwd_2rev.fa', 'a') as outfile:
                lines1= f1.readlines()
                lines2= f2.readlines()
                loc= lines1[0::4]
                desired_lines1= lines1[1::4]
                desired_lines2= lines2[1::4]
                for i in range(len(desired_lines1)):
                    reads= '>' + str(loc[i]) + desired_lines1[i].strip() + 'NNNNNNNN' + Seq(desired_lines2[i].strip()).reverse_complement()
                    print(reads, file=outfile)

In [12]:
join_files_1fwd('Seq1_1.fq', 'Seq1_2.fq')

In [13]:
#joins files assuming all reads from FASTQ file 2 are the forward read
def join_files_2fwd(file1, file2):
    with open(file2) as f2:
        with open(file1) as f1:
            with open ('joined_2fwd_1rev.fa', 'a') as outfile:
                lines1= f1.readlines()
                lines2= f2.readlines()
                loc= lines1[0::4]
                desired_lines1= lines1[1::4]
                desired_lines2= lines2[1::4]
                for i in range(len(desired_lines1)):
                    reads= '>' + str(loc[i]) + desired_lines2[i].strip() + 'NNNNNNNN' + Seq(desired_lines1[i].strip()).reverse_complement()
                    print(reads, file=outfile)

In [15]:
join_files_2fwd('Seq1_1.fq', 'Seq1_2.fq')

In [16]:
# creates a file with only correctly oriented (5' -> 3') reads
def cleaned_pairs(file1, file2):
    #picks the reads from file 1 that have a forward barcode and sends them to a "cleaned" file
    with open('joined_cleaned.fa', 'a') as output:
        for record in SeqIO.parse('joined_1fwd_2rev.fa', 'fasta'):
            seq= record.seq
            loc= record.id
            for k,v in top_barcode_list.items():
                if str(v) in seq:
                    print('>' + loc+ '\n' + str(seq), file= output)
    #picks the reads from file 2 that have a forward barcode and adds them to the same "cleaned" file
    with open('joined_cleaned.fa', 'a') as output:
        for record in SeqIO.parse('joined_2fwd_1rev.fa', 'fasta'):
            seq= record.seq
            loc= record.id
            for k,v in top_barcode_list.items():
                if str(v) in seq:
                    print('>' + loc+ '\n' + str(seq), file= output)

In [17]:
cleaned_pairs('Seq1_1.fq', 'Seq1_2.fq')

In [22]:
# sorts reads into separate fasta files, based on top barcode in dictionary that was created above
def parse_sort(file, barcode_list):
    for record in SeqIO.parse(file, 'fasta'):
        seq= str(record.seq)
        loc= record.id
        for k,v in barcode_list.items():
            if str(v) in seq:
                with open('%s.fasta'%str(k), 'a') as outputfile:
                    print('>' + loc + '\n' + seq, file=outputfile)

In [23]:
parse_sort('joined_cleaned.fa', top_barcode_list)

In [24]:
# in order to count the occurence of each amino acid, we have to import the genetic code
# assign genetic code to two dictionaries, one for each mutated active site position
gencode1 = { 
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M', 
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T', 
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K', 
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                  
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L', 
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P', 
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q', 
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R', 
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V', 
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A', 
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E', 
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G', 
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S', 
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L', 
        'TAC':'Y', 'TAT':'Y', 'TGC':'C', 'TGT':'C', 
        'TGG':'W'} 
gencode2 = { 
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M', 
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T', 
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K', 
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                  
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L', 
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P', 
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q', 
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R', 
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V', 
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A', 
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E', 
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G', 
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S', 
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L', 
        'TAC':'Y', 'TAT':'Y', 'TGC':'C', 'TGT':'C', 
        'TGG':'W'} 

In [25]:
# function below counts the occurence of each amino acid at the desired position based on input REGEX code
# NSfile, A3file, and C2file are the naive, ampicillin-selected, and cefotaxime-selected libraries, respectively
# DNAcode input is the REGEX code used to search for the desired codon position
# REGEX code must include 12-bp on either side of the desired codon
# and designated range of base pairs that occur between the two codons
def count_csv(NSfile, A3file, C2file, DNAcode):
    with open('%s'%'fullgencode-counts-' + str(NSfile[0:-13]) + '.csv', 'a') as outfile:
    
        #NS library
        listNS= ['A:A', 'A:R', 'A:N', 'A:D', 'A:C', 'A:Q', 'A:E', 'A:G', 'A:H', 'A:I', 'A:L', 'A:K', 'A:M', 'A:F', 'A:P', 'A:S', 'A:T', 'A:W', 'A:Y', 'A:V', 'R:A', 'R:R', 'R:N', 'R:D', 'R:C', 'R:Q', 'R:E', 'R:G', 'R:H', 'R:I', 'R:L', 'R:K', 'R:M', 'R:F', 'R:P', 'R:S', 'R:T', 'R:W', 'R:Y', 'R:V', 'N:A', 'N:R', 'N:N', 'N:D', 'N:C', 'N:Q', 'N:E', 'N:G', 'N:H', 'N:I', 'N:L', 'N:K', 'N:M', 'N:F', 'N:P', 'N:S', 'N:T', 'N:W', 'N:Y', 'N:V', 'D:A', 'D:R', 'D:N', 'D:D', 'D:C', 'D:Q', 'D:E', 'D:G', 'D:H', 'D:I', 'D:L', 'D:K', 'D:M', 'D:F', 'D:P', 'D:S', 'D:T', 'D:W', 'D:Y', 'D:V', 'C:A', 'C:R', 'C:N', 'C:D', 'C:C', 'C:Q', 'C:E', 'C:G', 'C:H', 'C:I', 'C:L', 'C:K', 'C:M', 'C:F', 'C:P', 'C:S', 'C:T', 'C:W', 'C:Y', 'C:V', 'Q:A', 'Q:R', 'Q:N', 'Q:D', 'Q:C', 'Q:Q', 'Q:E', 'Q:G', 'Q:H', 'Q:I', 'Q:L', 'Q:K', 'Q:M', 'Q:F', 'Q:P', 'Q:S', 'Q:T', 'Q:W', 'Q:Y', 'Q:V', 'E:A', 'E:R', 'E:N', 'E:D', 'E:C', 'E:Q', 'E:E', 'E:G', 'E:H', 'E:I', 'E:L', 'E:K', 'E:M', 'E:F', 'E:P', 'E:S', 'E:T', 'E:W', 'E:Y', 'E:V', 'G:A', 'G:R', 'G:N', 'G:D', 'G:C', 'G:Q', 'G:E', 'G:G', 'G:H', 'G:I', 'G:L', 'G:K', 'G:M', 'G:F', 'G:P', 'G:S', 'G:T', 'G:W', 'G:Y', 'G:V', 'H:A', 'H:R', 'H:N', 'H:D', 'H:C', 'H:Q', 'H:E', 'H:G', 'H:H', 'H:I', 'H:L', 'H:K', 'H:M', 'H:F', 'H:P', 'H:S', 'H:T', 'H:W', 'H:Y', 'H:V', 'I:A', 'I:R', 'I:N', 'I:D', 'I:C', 'I:Q', 'I:E', 'I:G', 'I:H', 'I:I', 'I:L', 'I:K', 'I:M', 'I:F', 'I:P', 'I:S', 'I:T', 'I:W', 'I:Y', 'I:V', 'L:A', 'L:R', 'L:N', 'L:D', 'L:C', 'L:Q', 'L:E', 'L:G', 'L:H', 'L:I', 'L:L', 'L:K', 'L:M', 'L:F', 'L:P', 'L:S', 'L:T', 'L:W', 'L:Y', 'L:V', 'K:A', 'K:R', 'K:N', 'K:D', 'K:C', 'K:Q', 'K:E', 'K:G', 'K:H', 'K:I', 'K:L', 'K:K', 'K:M', 'K:F', 'K:P', 'K:S', 'K:T', 'K:W', 'K:Y', 'K:V', 'M:A', 'M:R', 'M:N', 'M:D', 'M:C', 'M:Q', 'M:E', 'M:G', 'M:H', 'M:I', 'M:L', 'M:K', 'M:M', 'M:F', 'M:P', 'M:S', 'M:T', 'M:W', 'M:Y', 'M:V', 'F:A', 'F:R', 'F:N', 'F:D', 'F:C', 'F:Q', 'F:E', 'F:G', 'F:H', 'F:I', 'F:L', 'F:K', 'F:M', 'F:F', 'F:P', 'F:S', 'F:T', 'F:W', 'F:Y', 'F:V', 'P:A', 'P:R', 'P:N', 'P:D', 'P:C', 'P:Q', 'P:E', 'P:G', 'P:H', 'P:I', 'P:L', 'P:K', 'P:M', 'P:F', 'P:P', 'P:S', 'P:T', 'P:W', 'P:Y', 'P:V', 'S:A', 'S:R', 'S:N', 'S:D', 'S:C', 'S:Q', 'S:E', 'S:G', 'S:H', 'S:I', 'S:L', 'S:K', 'S:M', 'S:F', 'S:P', 'S:S', 'S:T', 'S:W', 'S:Y', 'S:V', 'T:A', 'T:R', 'T:N', 'T:D', 'T:C', 'T:Q', 'T:E', 'T:G', 'T:H', 'T:I', 'T:L', 'T:K', 'T:M', 'T:F', 'T:P', 'T:S', 'T:T', 'T:W', 'T:Y', 'T:V', 'W:A', 'W:R', 'W:N', 'W:D', 'W:C', 'W:Q', 'W:E', 'W:G', 'W:H', 'W:I', 'W:L', 'W:K', 'W:M', 'W:F', 'W:P', 'W:S', 'W:T', 'W:W', 'W:Y', 'W:V', 'Y:A', 'Y:R', 'Y:N', 'Y:D', 'Y:C', 'Y:Q', 'Y:E', 'Y:G', 'Y:H', 'Y:I', 'Y:L', 'Y:K', 'Y:M', 'Y:F', 'Y:P', 'Y:S', 'Y:T', 'Y:W', 'Y:Y', 'Y:V', 'V:A', 'V:R', 'V:N', 'V:D', 'V:C', 'V:Q', 'V:E', 'V:G', 'V:H', 'V:I', 'V:L', 'V:K', 'V:M', 'V:F', 'V:P', 'V:S', 'V:T', 'V:W', 'V:Y', 'V:V']
        for record in SeqIO.parse(NSfile, 'fasta'):
            seq=str(record.seq)
            if re.search(DNAcode, seq):
                x=re.search(DNAcode, seq)
                mut1= str(x.group()[12:15])
                mut2= str(x.group()[-15:-12])
                for k1,v1 in gencode1.items():
                    for k2,v2 in gencode2  .items():
                        if str(k1) in mut1 and str(k2) in mut2:
                            listNS.append(gencode1[mut1] + ':' + gencode2[mut2])
        dNS= {}
        for item in listNS:
            dNS[item]= dNS.get(item,0) + 1
        print('NS done!')

        
        #A3 library
        listA3= ['A:A', 'A:R', 'A:N', 'A:D', 'A:C', 'A:Q', 'A:E', 'A:G', 'A:H', 'A:I', 'A:L', 'A:K', 'A:M', 'A:F', 'A:P', 'A:S', 'A:T', 'A:W', 'A:Y', 'A:V', 'R:A', 'R:R', 'R:N', 'R:D', 'R:C', 'R:Q', 'R:E', 'R:G', 'R:H', 'R:I', 'R:L', 'R:K', 'R:M', 'R:F', 'R:P', 'R:S', 'R:T', 'R:W', 'R:Y', 'R:V', 'N:A', 'N:R', 'N:N', 'N:D', 'N:C', 'N:Q', 'N:E', 'N:G', 'N:H', 'N:I', 'N:L', 'N:K', 'N:M', 'N:F', 'N:P', 'N:S', 'N:T', 'N:W', 'N:Y', 'N:V', 'D:A', 'D:R', 'D:N', 'D:D', 'D:C', 'D:Q', 'D:E', 'D:G', 'D:H', 'D:I', 'D:L', 'D:K', 'D:M', 'D:F', 'D:P', 'D:S', 'D:T', 'D:W', 'D:Y', 'D:V', 'C:A', 'C:R', 'C:N', 'C:D', 'C:C', 'C:Q', 'C:E', 'C:G', 'C:H', 'C:I', 'C:L', 'C:K', 'C:M', 'C:F', 'C:P', 'C:S', 'C:T', 'C:W', 'C:Y', 'C:V', 'Q:A', 'Q:R', 'Q:N', 'Q:D', 'Q:C', 'Q:Q', 'Q:E', 'Q:G', 'Q:H', 'Q:I', 'Q:L', 'Q:K', 'Q:M', 'Q:F', 'Q:P', 'Q:S', 'Q:T', 'Q:W', 'Q:Y', 'Q:V', 'E:A', 'E:R', 'E:N', 'E:D', 'E:C', 'E:Q', 'E:E', 'E:G', 'E:H', 'E:I', 'E:L', 'E:K', 'E:M', 'E:F', 'E:P', 'E:S', 'E:T', 'E:W', 'E:Y', 'E:V', 'G:A', 'G:R', 'G:N', 'G:D', 'G:C', 'G:Q', 'G:E', 'G:G', 'G:H', 'G:I', 'G:L', 'G:K', 'G:M', 'G:F', 'G:P', 'G:S', 'G:T', 'G:W', 'G:Y', 'G:V', 'H:A', 'H:R', 'H:N', 'H:D', 'H:C', 'H:Q', 'H:E', 'H:G', 'H:H', 'H:I', 'H:L', 'H:K', 'H:M', 'H:F', 'H:P', 'H:S', 'H:T', 'H:W', 'H:Y', 'H:V', 'I:A', 'I:R', 'I:N', 'I:D', 'I:C', 'I:Q', 'I:E', 'I:G', 'I:H', 'I:I', 'I:L', 'I:K', 'I:M', 'I:F', 'I:P', 'I:S', 'I:T', 'I:W', 'I:Y', 'I:V', 'L:A', 'L:R', 'L:N', 'L:D', 'L:C', 'L:Q', 'L:E', 'L:G', 'L:H', 'L:I', 'L:L', 'L:K', 'L:M', 'L:F', 'L:P', 'L:S', 'L:T', 'L:W', 'L:Y', 'L:V', 'K:A', 'K:R', 'K:N', 'K:D', 'K:C', 'K:Q', 'K:E', 'K:G', 'K:H', 'K:I', 'K:L', 'K:K', 'K:M', 'K:F', 'K:P', 'K:S', 'K:T', 'K:W', 'K:Y', 'K:V', 'M:A', 'M:R', 'M:N', 'M:D', 'M:C', 'M:Q', 'M:E', 'M:G', 'M:H', 'M:I', 'M:L', 'M:K', 'M:M', 'M:F', 'M:P', 'M:S', 'M:T', 'M:W', 'M:Y', 'M:V', 'F:A', 'F:R', 'F:N', 'F:D', 'F:C', 'F:Q', 'F:E', 'F:G', 'F:H', 'F:I', 'F:L', 'F:K', 'F:M', 'F:F', 'F:P', 'F:S', 'F:T', 'F:W', 'F:Y', 'F:V', 'P:A', 'P:R', 'P:N', 'P:D', 'P:C', 'P:Q', 'P:E', 'P:G', 'P:H', 'P:I', 'P:L', 'P:K', 'P:M', 'P:F', 'P:P', 'P:S', 'P:T', 'P:W', 'P:Y', 'P:V', 'S:A', 'S:R', 'S:N', 'S:D', 'S:C', 'S:Q', 'S:E', 'S:G', 'S:H', 'S:I', 'S:L', 'S:K', 'S:M', 'S:F', 'S:P', 'S:S', 'S:T', 'S:W', 'S:Y', 'S:V', 'T:A', 'T:R', 'T:N', 'T:D', 'T:C', 'T:Q', 'T:E', 'T:G', 'T:H', 'T:I', 'T:L', 'T:K', 'T:M', 'T:F', 'T:P', 'T:S', 'T:T', 'T:W', 'T:Y', 'T:V', 'W:A', 'W:R', 'W:N', 'W:D', 'W:C', 'W:Q', 'W:E', 'W:G', 'W:H', 'W:I', 'W:L', 'W:K', 'W:M', 'W:F', 'W:P', 'W:S', 'W:T', 'W:W', 'W:Y', 'W:V', 'Y:A', 'Y:R', 'Y:N', 'Y:D', 'Y:C', 'Y:Q', 'Y:E', 'Y:G', 'Y:H', 'Y:I', 'Y:L', 'Y:K', 'Y:M', 'Y:F', 'Y:P', 'Y:S', 'Y:T', 'Y:W', 'Y:Y', 'Y:V', 'V:A', 'V:R', 'V:N', 'V:D', 'V:C', 'V:Q', 'V:E', 'V:G', 'V:H', 'V:I', 'V:L', 'V:K', 'V:M', 'V:F', 'V:P', 'V:S', 'V:T', 'V:W', 'V:Y', 'V:V']
        for record in SeqIO.parse(A3file, 'fasta'):
            seq=str(record.seq)
            if re.search(DNAcode, seq):
                x=re.search(DNAcode, seq)
                mut1= str(x.group()[12:15])
                mut2= str(x.group()[-15:-12])
                for k1,v1 in gencode1.items():
                    for k2,v2 in gencode2.items():
                        if str(k1) in mut1 and str(k2) in mut2:
                            listA3.append(gencode1[mut1] + ':' + gencode2[mut2])
        dA3= {}
        for item in listA3:
            dA3[item]= dA3.get(item,0) + 1
        print('A3 done!')
        
        
        #C2 library
        listC2= ['A:A', 'A:R', 'A:N', 'A:D', 'A:C', 'A:Q', 'A:E', 'A:G', 'A:H', 'A:I', 'A:L', 'A:K', 'A:M', 'A:F', 'A:P', 'A:S', 'A:T', 'A:W', 'A:Y', 'A:V', 'R:A', 'R:R', 'R:N', 'R:D', 'R:C', 'R:Q', 'R:E', 'R:G', 'R:H', 'R:I', 'R:L', 'R:K', 'R:M', 'R:F', 'R:P', 'R:S', 'R:T', 'R:W', 'R:Y', 'R:V', 'N:A', 'N:R', 'N:N', 'N:D', 'N:C', 'N:Q', 'N:E', 'N:G', 'N:H', 'N:I', 'N:L', 'N:K', 'N:M', 'N:F', 'N:P', 'N:S', 'N:T', 'N:W', 'N:Y', 'N:V', 'D:A', 'D:R', 'D:N', 'D:D', 'D:C', 'D:Q', 'D:E', 'D:G', 'D:H', 'D:I', 'D:L', 'D:K', 'D:M', 'D:F', 'D:P', 'D:S', 'D:T', 'D:W', 'D:Y', 'D:V', 'C:A', 'C:R', 'C:N', 'C:D', 'C:C', 'C:Q', 'C:E', 'C:G', 'C:H', 'C:I', 'C:L', 'C:K', 'C:M', 'C:F', 'C:P', 'C:S', 'C:T', 'C:W', 'C:Y', 'C:V', 'Q:A', 'Q:R', 'Q:N', 'Q:D', 'Q:C', 'Q:Q', 'Q:E', 'Q:G', 'Q:H', 'Q:I', 'Q:L', 'Q:K', 'Q:M', 'Q:F', 'Q:P', 'Q:S', 'Q:T', 'Q:W', 'Q:Y', 'Q:V', 'E:A', 'E:R', 'E:N', 'E:D', 'E:C', 'E:Q', 'E:E', 'E:G', 'E:H', 'E:I', 'E:L', 'E:K', 'E:M', 'E:F', 'E:P', 'E:S', 'E:T', 'E:W', 'E:Y', 'E:V', 'G:A', 'G:R', 'G:N', 'G:D', 'G:C', 'G:Q', 'G:E', 'G:G', 'G:H', 'G:I', 'G:L', 'G:K', 'G:M', 'G:F', 'G:P', 'G:S', 'G:T', 'G:W', 'G:Y', 'G:V', 'H:A', 'H:R', 'H:N', 'H:D', 'H:C', 'H:Q', 'H:E', 'H:G', 'H:H', 'H:I', 'H:L', 'H:K', 'H:M', 'H:F', 'H:P', 'H:S', 'H:T', 'H:W', 'H:Y', 'H:V', 'I:A', 'I:R', 'I:N', 'I:D', 'I:C', 'I:Q', 'I:E', 'I:G', 'I:H', 'I:I', 'I:L', 'I:K', 'I:M', 'I:F', 'I:P', 'I:S', 'I:T', 'I:W', 'I:Y', 'I:V', 'L:A', 'L:R', 'L:N', 'L:D', 'L:C', 'L:Q', 'L:E', 'L:G', 'L:H', 'L:I', 'L:L', 'L:K', 'L:M', 'L:F', 'L:P', 'L:S', 'L:T', 'L:W', 'L:Y', 'L:V', 'K:A', 'K:R', 'K:N', 'K:D', 'K:C', 'K:Q', 'K:E', 'K:G', 'K:H', 'K:I', 'K:L', 'K:K', 'K:M', 'K:F', 'K:P', 'K:S', 'K:T', 'K:W', 'K:Y', 'K:V', 'M:A', 'M:R', 'M:N', 'M:D', 'M:C', 'M:Q', 'M:E', 'M:G', 'M:H', 'M:I', 'M:L', 'M:K', 'M:M', 'M:F', 'M:P', 'M:S', 'M:T', 'M:W', 'M:Y', 'M:V', 'F:A', 'F:R', 'F:N', 'F:D', 'F:C', 'F:Q', 'F:E', 'F:G', 'F:H', 'F:I', 'F:L', 'F:K', 'F:M', 'F:F', 'F:P', 'F:S', 'F:T', 'F:W', 'F:Y', 'F:V', 'P:A', 'P:R', 'P:N', 'P:D', 'P:C', 'P:Q', 'P:E', 'P:G', 'P:H', 'P:I', 'P:L', 'P:K', 'P:M', 'P:F', 'P:P', 'P:S', 'P:T', 'P:W', 'P:Y', 'P:V', 'S:A', 'S:R', 'S:N', 'S:D', 'S:C', 'S:Q', 'S:E', 'S:G', 'S:H', 'S:I', 'S:L', 'S:K', 'S:M', 'S:F', 'S:P', 'S:S', 'S:T', 'S:W', 'S:Y', 'S:V', 'T:A', 'T:R', 'T:N', 'T:D', 'T:C', 'T:Q', 'T:E', 'T:G', 'T:H', 'T:I', 'T:L', 'T:K', 'T:M', 'T:F', 'T:P', 'T:S', 'T:T', 'T:W', 'T:Y', 'T:V', 'W:A', 'W:R', 'W:N', 'W:D', 'W:C', 'W:Q', 'W:E', 'W:G', 'W:H', 'W:I', 'W:L', 'W:K', 'W:M', 'W:F', 'W:P', 'W:S', 'W:T', 'W:W', 'W:Y', 'W:V', 'Y:A', 'Y:R', 'Y:N', 'Y:D', 'Y:C', 'Y:Q', 'Y:E', 'Y:G', 'Y:H', 'Y:I', 'Y:L', 'Y:K', 'Y:M', 'Y:F', 'Y:P', 'Y:S', 'Y:T', 'Y:W', 'Y:Y', 'Y:V', 'V:A', 'V:R', 'V:N', 'V:D', 'V:C', 'V:Q', 'V:E', 'V:G', 'V:H', 'V:I', 'V:L', 'V:K', 'V:M', 'V:F', 'V:P', 'V:S', 'V:T', 'V:W', 'V:Y', 'V:V']
        for record in SeqIO.parse(C2file, 'fasta'):
            seq=str(record.seq)
            if re.search(DNAcode, seq):
                x=re.search(DNAcode, seq)
                mut1= str(x.group()[12:15])
                mut2= str(x.group()[-15:-12])
                for k1,v1 in gencode1.items():
                    for k2,v2 in gencode2.items():
                        if str(k1) in mut1 and str(k2) in mut2:
                            listC2.append(gencode1[mut1] + ':' + gencode2[mut2])
        dC2= {}
        for item in listC2:
            dC2[item]= dC2.get(item,0) + 1
        print('C2 done!')
        
        #write to outfile
        fieldnames= ['AA', 'NS', 'A3', 'C2']
        writer=csv.writer(outfile)
        writer.writerow(fieldnames)
        
        for k,v in dNS.items():
            print(str(k) + ',' + str(v) + ',' + str(dA3[k]) + ',' + str(dC2[k]), file=outfile)

In [ ]:
# each of the 136 libraries (for each pair of positions) was processed individually
# below is one example of two distant active site positions, K73X-K234X
count_csv('K73X-K234X-ns-top.fasta',  
          'K73X-K234X-A3-top.fasta',
          'K73X-K234X-C2-top.fasta', 
          r'TGCAGTACCAGT[ACTG]{3}GTTATGGCGGCC[ACTGN]{30,200}ACTGTGGGTGAT[ACTG]{3}ACCGGCAGCGGC')

In [ ]:
# below is an example of two nearby active site positions, K234X-G236X
count_csv('K234X-G236X-ns-top.fasta',
         'K234X-G236X-A3-top.fasta',
         'K234X-G236X-C2-top.fasta',
          r'ACTGTGGGTGAT[ACTG]{3}ACC[ACTG]{3}AGCGGCGACTAC')